## **1. Import pandas as pd**


Csv file loaded into pandas dataframe


First 10 rows of the dataset shown

In [38]:
import pandas as pd

house_sales = pd.read_csv("kc_house_data.csv")
print(house_sales[:10])


           id             date      price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000   221900.0         3       1.00         1180   
1  6414100192  20141209T000000   538000.0         3       2.25         2570   
2  5631500400  20150225T000000   180000.0         2       1.00          770   
3  2487200875  20141209T000000   604000.0         4       3.00         1960   
4  1954400510  20150218T000000   510000.0         3       2.00         1680   
5  7237550310  20140512T000000  1225000.0         4       4.50         5420   
6  1321400060  20140627T000000   257500.0         3       2.25         1715   
7  2008000270  20150115T000000   291850.0         3       1.50         1060   
8  2414600126  20150415T000000   229500.0         3       1.00         1780   
9  3793500160  20150312T000000   323000.0         3       2.50         1890   

   sqft_lot  floors  waterfront  view  ...  grade  sqft_above  sqft_basement  \
0      5650     1.0           0     0  ...      7 

## **2. Displaying basic information about house_sales dataset**

In [39]:
house_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

## **3. Data Cleaning**

#### 3.1. Identifying and handling missing values

From the previous code snippet where the description of the dataframe is shown, we can see that there are a total of 21613 rows of data. Out of the 21613, there are no null values in the columns.

Some columns have the value zero, which explains that the house in question doesn't have that specific feature. So, if the year of renovation is zero, it means the house sold was never renovated.

So, there not being any missing values, I will keep the dataframe unchanged.

#### 3.2. Checking and removing duplicate records

For checking for duplicates, I mainly want to focus on - id, zipcodes and prices.


The id is the unique identifier for each house. So, I want to check first if duplicate ids exist

In [40]:
check_duplicate_ids = house_sales.duplicated('id')
print(check_duplicate_ids)

0        False
1        False
2        False
3        False
4        False
         ...  
21608    False
21609    False
21610    False
21611    False
21612    False
Length: 21613, dtype: bool


Since this doesn't clearly give an idea, I want to count each instance of the ids

In [41]:
number_of_ids = house_sales["id"].value_counts()
print(number_of_ids)

id
795000620     3
8651510380    2
2568300045    2
9353300600    2
4139480200    2
             ..
2767603649    1
1446403617    1
5602000275    1
1786200010    1
1523300157    1
Name: count, Length: 21436, dtype: int64


Here we can see that there are ids that are repeated but before removing them, I want to see if they have different zipcodes. although it's not possible for same id to occur through different locations

In [42]:
duplicate_ids = house_sales[house_sales.duplicated("id", keep= False)]
id_group_zipcode = duplicate_ids.groupby('id')['zipcode'].apply(list).reset_index()
print(id_group_zipcode)


             id         zipcode
0       1000102  [98002, 98002]
1       7200179  [98055, 98055]
2     109200390  [98023, 98023]
3     123039336  [98106, 98106]
4     251300110  [98003, 98003]
..          ...             ...
171  9407110710  [98045, 98045]
172  9809000020  [98004, 98004]
173  9828200460  [98122, 98122]
174  9834200305  [98144, 98144]
175  9834200885  [98144, 98144]

[176 rows x 2 columns]


Here I used duplicate_ids to list the ids that are repeated and then grouped those ids to see the corresponding zipcodes

So, for each repeated id, the zipcodes are the same, that means that there are same house information repeated. But we have another variable called price. If price for those repeated ids are same then the rows corresponding to those ids will be removed, and if the prices are different, then it means that the same house was sold again, in which case that entry will be kept.

In [43]:
duplicate_ids = house_sales[house_sales.duplicated("id", keep=False)]
price_check = duplicate_ids.groupby('id')['price'].nunique().reset_index()
price_check.columns = ['id', 'unique_price_count']
same_price_ids = price_check[price_check['unique_price_count'] == 1]['id']
house_sales_filtered = house_sales[~((house_sales['id'].isin(same_price_ids)) & (house_sales.duplicated('id', keep='first')))]


print(house_sales_filtered)

               id             date     price  bedrooms  bathrooms  \
0      7129300520  20141013T000000  221900.0         3       1.00   
1      6414100192  20141209T000000  538000.0         3       2.25   
2      5631500400  20150225T000000  180000.0         2       1.00   
3      2487200875  20141209T000000  604000.0         4       3.00   
4      1954400510  20150218T000000  510000.0         3       2.00   
...           ...              ...       ...       ...        ...   
21608   263000018  20140521T000000  360000.0         3       2.50   
21609  6600060120  20150223T000000  400000.0         4       2.50   
21610  1523300141  20140623T000000  402101.0         2       0.75   
21611   291310100  20150116T000000  400000.0         3       2.50   
21612  1523300157  20141015T000000  325000.0         2       0.75   

       sqft_living  sqft_lot  floors  waterfront  view  ...  grade  \
0             1180      5650     1.0           0     0  ...      7   
1             2570      7242   

- duplicate_ids gives the ids that are duplicate

- price_check gives a dataframe of duplicate ids with how many unique price each one has

- price_check.column sets the column names for price_check

- same_price_ids gives the ids that have a single price and house_sales_filtered then removes those rows with duplicate ids that have the same price

#### 3.3. Converting date column with a proper date-time format 

In [45]:
print(house_sales_filtered["date"].head(10))

0    20141013T000000
1    20141209T000000
2    20150225T000000
3    20141209T000000
4    20150218T000000
5    20140512T000000
6    20140627T000000
7    20150115T000000
8    20150415T000000
9    20150312T000000
Name: date, dtype: object
